<a href="https://colab.research.google.com/github/alesaccoia/IULM_DDM2324_Notebooks/blob/main/32_fine_tuning_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning di un modello di classificazione

In questo notebook vediamo come addestrare un modello pre-trained, in questo caso https://huggingface.co/dbmdz/bert-base-italian-xxl-cased .

**Nota che per avere un tempo di esecuzione gestibile bisogna comprare dei crediti e attaccare una GPU, per esempio una T4**

Iniziamo installando tutte le librerie richieste (sono tutte relative a Huggingface)


In [ ]:
!pip install huggingface
!pip install datasets
!pip install evaluate
!pip install accelerate
!pip install transformers[torch]

## Caricamento del dataset

In questa sezione carichiamo il dataset e etichettiamo come Positive le reviews con una valutazione da 4 a 5 stelle, come negative le reviews da 1 a 2 stelle

In [6]:
import pandas as pd
from datasets import Dataset

# Load the data
df = pd.read_csv("italian_reviews.txt")

# Filter out reviews with 3 stars
df = df[df['review_stars'] != 3]


# Define the sentiment based on the review stars
df['label'] = df['review_stars'].apply(lambda x: 1 if x >= 4 else 0)

# Select the relevant columns
df = df[['review_text', 'label']]



## Conversione a un Dataset di HuggingFace

In [7]:
# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)


## Riduzione del numero di esempi

Per limitare il tempo di fine tuning, creiamo due  dataset bilanciati (train + eval) con 2000 esempi ciascuno di reviews positive e negative

In [8]:
from datasets import concatenate_datasets

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

def create_stratified_dataset(dataset, num_samples_per_class):
    # Shuffle the dataset
    shuffled_dataset = dataset.shuffle(seed=42)

    # Separate positive and negative instances
    positive_dataset = shuffled_dataset.filter(lambda example: example['label'] == 1)
    negative_dataset = shuffled_dataset.filter(lambda example: example['label'] == 0)

    # Select 1000 instances from each class
    positive_subset = positive_dataset.select(range(num_samples_per_class))
    negative_subset = negative_dataset.select(range(num_samples_per_class))

    # Concatenate the subsets and shuffle
    balanced_dataset = concatenate_datasets([positive_subset, negative_subset]).shuffle(seed=42)
    return balanced_dataset

# Create a balanced dataset with 1000 instances of each class
balanced_dataset = create_stratified_dataset(dataset, 2000)

# Split the balanced dataset into training and testing sets
small_train_dataset, small_eval_dataset = balanced_dataset.train_test_split(test_size=0.5).values()



Filter:   0%|          | 0/143058 [00:00<?, ? examples/s]

Filter:   0%|          | 0/143058 [00:00<?, ? examples/s]

## Tokenizzazione

In questa fase si usa il tokenizzatore del modello prescelto, in modalitá blackbox, usando AutoTokenizer. Questo é comodo perché ogni modello può essere stato addestrato con un algoritmo di tokenizzaione (divisione in parole o caratteri) diverso, e vogliamo mantenerlo sia durante il fine tuning sia durante l'inferenza

In [9]:
from transformers import AutoTokenizer

model_checkpoint = "dbmdz/bert-base-italian-xxl-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["review_text"], padding="max_length", truncation=True)

small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## Caricamento di un modello pre-trained

Carichiamo il modello di linguaggio. Fare attenzione al "warning" che ci dice che questo modello necessita di fine tuning per essere utilizzato su di un task di classificazione

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-xxl-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Fine-Tuning

Utilizzando tutti i valori di default

In [11]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

## Valutazione

In [12]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

# questa funzione di valutazione calcola semplicemente la proporzione di etichette corrette
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Training


In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.102540,0.974000
2,0.160900,0.141284,0.970000
3,0.160900,0.129552,0.974500


TrainOutput(global_step=750, training_loss=0.11921665700276693, metrics={'train_runtime': 784.1214, 'train_samples_per_second': 7.652, 'train_steps_per_second': 0.956, 'total_flos': 1578666332160000.0, 'train_loss': 0.11921665700276693, 'epoch': 3.0})

## Creazione di una nuova pipeline e test di alcune frasi

In [23]:
import torch
from transformers import pipeline

test_sentences = [
    "Sono veramente soddisfatto!",
    "Questo prodotto è spazzatura.",
    "Velocissimi, consigliato",
    "Avrei preferito una consegna più veloce ma il prodotto è sicuramente di ottima qualità"
]

# Check if CUDA is available and set the device
device = 0 if torch.cuda.is_available() else -1

# Load the model and tokenizer into the pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)

# Test the model
outputs = classifier(test_sentences)
pd.DataFrame(outputs)


,label,score
0,LABEL_1,0.999597
1,LABEL_0,0.998525
2,LABEL_1,0.999623
3,LABEL_1,0.999520
